# Process Subdirectories

#### Updated: Dec 6, 2022

#  

Inspect '02_downloaded' folder for fully downloaded subdirectories, move these subdirectories to '03_processing'. Process subdirectories. Record processing performance in a 'process_log'. Then move processed subdirectories to '04_processed' folder. 

In [1]:
import pandas as pd
import os
import shutil
import time
from datetime import datetime
from random import randrange

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
process_on_VDI = True

rootDir_local = 'C:/Users/rnaidoo/Documents/Projects_data/Alouette_I/' #C: is not persistent on VDI
rootDir_U = 'U:/Data_Science/Projects_data/Alouette_I/'
downloadedDir = rootDir_local + '02_downloaded/'
processingDir = rootDir_local + '03_processing/'
if process_on_VDI:
    processedDir = rootDir_U + '04_processed/' 
    resultDir = rootDir_U + '05_result/' 
    logDir = '//scientific/L-MP-Data/Massive files/Python/rnaidoo/Alouette_I/' #DO NOT CHANGE
    move_to_U = True
else:
    processedDir = rootDir_local + '04_processed/' 
    resultDir = rootDir_local + '05_result/' 
    logDir = resultDir
    move_to_U = False

#  

#### Functions:

In [4]:
def move_images(old_dir, new_dir, roll, subdir, copy_to_other_drive=False):
    oldDir = old_dir + roll + '/' + subdir + '/'
    newDir = new_dir + roll + '/' + subdir + '/'
    os.makedirs(newDir, exist_ok=True)
    
    if copy_to_other_drive:
        for file in os.listdir(oldDir):
            shutil.copyfile(oldDir+file, newDir+file)
    else:
        for file in os.listdir(oldDir):
            os.rename(oldDir+file, newDir+file)
    
    shutil.rmtree(old_dir + roll + '/')

#  

#### Process new fully downloaded subdirectories:

Move one fully downloaded subdirectory from '02_downloaded' to '03_processing', process, then move images to '04_processing' - do this one subdirectory at a time. 

In [5]:
df_result = pd.DataFrame()
for roll in os.listdir(downloadedDir):
    if 'R' in roll:
        for subdirectory in os.listdir(downloadedDir + roll):
            start = time.time()
            subdir_path_end = roll + '/' + subdirectory + '/'
            
            #Move to '03_processing'
            move_images(old_dir=downloadedDir, new_dir=processingDir, roll=roll, subdir=subdirectory)
            
            #Process
            print('Processing ' + subdir_path_end + ' subdirectory...')
            !python scan2data/user_input.py $processingDir $resultDir

            #Consolidate results
            df_dot = pd.read_csv(resultDir + 'df_dot.csv')
            n_dot = len(df_dot)
            df_dot['processed_image_class'] = 'dot'
            
            df_num = pd.read_csv(resultDir + 'df_num.csv')
            n_num = len(df_num)
            df_num['processed_image_class'] = 'num'
            
            df_loss = pd.read_csv(resultDir + 'df_loss.csv')
            n_loss = len(df_loss)
            df_loss['processed_image_class'] = 'loss'
            
            df_outlier = pd.read_csv(resultDir + 'df_outlier.csv')
            n_outlier = len(df_outlier)
            df_outlier['processed_image_class'] = 'outlier'
            
            df_tot = pd.concat([df_dot, df_num, df_loss, df_outlier])
            df_tot['Roll'] = roll
            df_tot['Subdirectory'] = subdirectory
            df_tot = df_tot.drop(columns=['file_name', 'mapped_coord', 'subdir_name', 'raw', 'ionogram', 'raw_metadata', 
                                          'trimmed_metadata', 'padded', 'dilated_metadata'], errors='ignore')
            df_tot.to_csv(resultDir + 'result-' + roll + '_' + subdirectory + '.csv', index=False)
            
            os.remove(resultDir + 'df_dot.csv')
            os.remove(resultDir + 'df_num.csv')
            os.remove(resultDir + 'df_loss.csv')
            os.remove(resultDir + 'df_outlier.csv')
            
            end = time.time()
            t = end - start
            print('Processing time for subdirectory: ' + str(round(t/60, 1)) + ' min')
            
            #Record performance
            df_result_ = pd.DataFrame({
                'Roll': roll,
                'Subdirectory': subdirectory,
                'Images_processed': n_dot + n_num + n_loss + n_outlier,
                'Images_dot': n_dot,
                'Images_num': n_num,
                'Images_loss': n_loss,
                'Images_outlier': n_outlier,
                'Process_time': t,
                'Process_timestamp': datetime.fromtimestamp(end)
            }, index=[0])
            df_result = pd.concat([df_result, df_result_], axis=0, ignore_index=True)  
            
            #Move to '04_processed'
            print("Moving images to '04_processed'")
            move_images(old_dir=processingDir, new_dir=processedDir, roll=roll, subdir=subdirectory, copy_to_other_drive=move_to_U)

Processing R014207821/3342-A38/ subdirectory...
Processing time for subdirectory: 5.7 min
Moving images to '04_processed'


Add to results to 'process_log':

In [6]:
if os.path.exists(logDir + 'process_log.csv'):
    df_log = pd.read_csv(logDir + 'process_log.csv')
    df_update = pd.concat([df_log, df_result], axis=0, ignore_index=True)
    df_update.to_csv(logDir + 'process_log.csv', index=False)
else:
    if len(df_result) > 0:
        df_result.to_csv(logDir + 'process_log.csv', index=False)

Backup 'process_log' (10% of the time):

In [7]:
if randrange(10) == 7:
    df_log = pd.read_csv(logDir + 'process_log.csv')
    datetime_str = datetime.now().strftime("%Y%m%d_%Hh%M")
    os.makedirs(logDir + 'backups/', exist_ok=True)
    df_log.to_csv(logDir + 'backups/' + 'process_log-' + datetime_str + '.csv', index=False)

Processing on CSA laptop (C:) (Intel Core i7-10610U CPU @ 1.80 GHz, 2.30 GHz, 16 GB RAM) --> 18.4 min <br>
Processing on VDI HP (C:) (Intel Xeon Gold 6248 CPU @ 2.50GHz, 2.49 GHz  (2 processors), 64 GB RAM) --> 18.1 min <br>
<br>
- Upon inspection of computer performance, processing appears to be much more CPU-intensive rather than RAM-intensive